In [9]:
# Import necessary libraries
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import os

# Define your service account file path and your project ID
service_account_file = 'adamh-wedge-project-6f59b14d0763.json'
project_id = 'adamh-wedge-project'

# Define your credentials and your client
credentials = service_account.Credentials.from_service_account_file(service_account_file)
client = bigquery.Client(credentials=credentials, project=project_id)

In [10]:
# SQL query to extract data
sql_query = """
SELECT DATE(datetime) as date, 
       EXTRACT(HOUR FROM datetime) as hour, 
       SUM(total) as total_spend, 
       COUNT(DISTINCT CONCAT(CAST(datetime AS STRING), CAST(register_no AS STRING), CAST(emp_no AS STRING), CAST(trans_no AS STRING))) as transactions, 
       SUM(CASE WHEN trans_status IN ('V', 'R') THEN -1 ELSE 1 END) as items
FROM `adamh-wedge-project.wedge_transactions.*`
WHERE department NOT IN (0, 15) AND (trans_status IS NULL OR trans_status IN (' ','', 'V', 'R'))
GROUP BY date, hour
ORDER BY date, hour
"""
df_date_hour = client.query(sql_query).to_dataframe()

# Create SQLite table
import sqlite3

# Check if the database file exists and delete it if it does
if os.path.exists('sales.db'):
    os.remove('sales.db')

# Create a new SQLite database and connection
conn = sqlite3.connect('sales.db')

# Write the DataFrame to a new SQLite table
df_date_hour.to_sql('sales_by_date_by_hour', conn, if_exists='replace', index=False)

39330

In [11]:
# SQL query to extract data
sql_query = """
SELECT card_no, 
       EXTRACT(YEAR FROM datetime) as year, 
       EXTRACT(MONTH FROM datetime) as month, 
       SUM(total) as sales, 
       COUNT(DISTINCT CONCAT(CAST(datetime AS STRING), CAST(register_no AS STRING), CAST(emp_no AS STRING), CAST(trans_no AS STRING))) as transactions, 
       SUM(CASE WHEN trans_status IN ('V', 'R') THEN -1 ELSE 1 END) as items
FROM `adamh-wedge-project.wedge_transactions.*`
WHERE department NOT IN (0, 15) AND (trans_status IS NULL OR trans_status IN (' ','', 'V', 'R'))
GROUP BY card_no, year, month
ORDER BY card_no, year, month
"""
df_owner_year_month = client.query(sql_query).to_dataframe()

# Create SQLite table
df_owner_year_month.to_sql('sales_by_owner_by_year_by_month', conn, if_exists='replace', index=False)

808893

In [12]:
# SQL query to extract data
sql_query = """
SELECT upc, 
       description, 
       department, 
       EXTRACT(YEAR FROM datetime) as year, 
       EXTRACT(MONTH FROM datetime) as month, 
       SUM(total) as sales, 
       COUNT(DISTINCT CONCAT(CAST(datetime AS STRING), CAST(register_no AS STRING), CAST(emp_no AS STRING), CAST(trans_no AS STRING))) as transactions, 
       SUM(CASE WHEN trans_status IN ('V', 'R') THEN -1 ELSE 1 END) as items
FROM `adamh-wedge-project.wedge_transactions.*`
WHERE department NOT IN (0, 15) AND (trans_status IS NULL OR trans_status IN (' ','', 'V', 'R'))
GROUP BY upc, description, department, year, month
ORDER BY upc, description, department, year, month
"""
df_product_year_month = client.query(sql_query).to_dataframe()

# Create SQLite table
df_product_year_month.to_sql('sales_by_product_by_year_by_month', conn, if_exists='replace', index=False)

# Close the connection
conn.close()